In [ ]:
import pandas as pd
import numpy as np

#Creating an empty Dataframe with 12 rows, one for each experiment. Fluor_conc is the theoretical Fluoroscein concentration.
blank_rows = np.arange(1,12,1)
fluor_conc = []
for i in range(0,11):
    fluor_conc.append(10/(2**i))
row_data = {
    'Row' : blank_rows,
    'Fluor_conc' : fluor_conc
}
blank_corrected = pd.DataFrame(row_data)




In [ ]:
#Importing data from the Excel sheets
dfs = {}
for i in range(1,13):
    dfs[f"df{i}"] = pd.read_excel(
        io = '/Users/wilsonporteus/Documents/GitHub/ssb_s5/Main Assesment/Data/pb'+str(i)+'.xlsx',
        #change this for your data file path
        header=0,
        sheet_name = 'End point', usecols="B:L",
        skiprows=13,
        nrows=9, engine="openpyxl"
        )
    avg = []
    for col in range(0,11):
        avg.append(
            np.average(dfs[f"df{i}"].iloc[:,col])
        )
    blank_corrected.insert(loc=i+1, column=f"df{i}_bc", value=avg)

In [ ]:
#Getting Log Data: Log Fluor_conc and Log Data
log_df = np.log10(blank_corrected)


In [ ]:
#Determining the Fit (Gradient of Log-Log trendline) and R^2 of the trendline.
slopes = []
r2_vals = []
x = log_df["Fluor_conc"]

for col in range(2,14):
    y = log_df.iloc[:,col]
    m, b = np.polyfit(x, y, 1)
    y_pred = m * x + b
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = 1 - (ss_res / ss_tot)
    slopes.append(m)
    r2_vals.append(r2)

exp_number = np.arange(1,13,1)
final_data = {
    "Expirement Number" : exp_number,
    "Gradient" : slopes,
    "R2" : r2_vals
}
final_df = pd.DataFrame(final_data)

final_df.to_excel("PB_gradient.xlsx")